In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sns

In [2]:
app_tr = pd.read_csv('application_train.csv')

In [3]:
y = app_tr['TARGET'].values
app = pd.read_csv('ready_for_model_building_appl.csv')
del app_tr

In [4]:
y.shape

(307511,)

In [5]:
selected_feats = ['CREDIT_INCOME_RATIO', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_ENDDATE_FACT', 'CNT_INSTALMENT', 'DAYS_REGISTRATION','DAYS_LAST_DUE','DAYS_FIRST_DUE','AMT_ANNUITY_x','NAME_EDUCATION_TYPE_Higher education','ENTIRE_INCOME_PER_CHILD','AMT_CREDIT','LATE_PAYMENT_DAYS','ANNUITY_INCOME_RATIO','ENTIRE_INCOME_PER_FAM_MEMBER', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH','HC','SCOFR','MONTHS_BALANCE_x', 'CNT_INSTALMENT_FUTURE','SK_DPD_DEF_x','DAYS_INSTALMENT','Closed','PERCENT_COMPLETION','AMT_GOODS_PRICE','ENTIRE_INCOME', 'AMT_INSTALMENT','CNT_DRAWINGS_CURRENT', 'DAYS_ENTRY_PAYMENT', 'DAYS_LAST_PHONE_CHANGE','DAYS_CREDIT_ENDDATE', 'CREDIT_GOODS_PRICE_DIFF','Active_x','DAYS_CREDIT_UPDATE', 'CNT_DRAWINGS_ATM_CURRENT','AMT_PAYMENT','ANNUITY_ENTIRE_INCOME_RATIO', 'DAYS_BIRTH', 'XAP', 'CREDIT_ENTIRE_INCOME_RATIO', 'YOKIAN_SCORE', 'DAYS_CREDIT', 'EXT_SOURCE_1', 'EXT_SOURCE_3', 'EXT_SOURCE_2']

In [6]:
len(selected_feats)

47

In [23]:
X2 = app[selected_feats].values
X2_sub = X2[y.shape[0]:, :]
X2 = X2[:y.shape[0], :]

In [9]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [24]:
scaler2 = StandardScaler()
scaler2.fit(app[selected_feats].values)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [25]:
X2 = scaler2.transform(X2)
X2_sub = scaler2.transform(X2_sub)

In [26]:
X2_tr, X2_ts, y_tr, y_ts = train_test_split(X2, y, test_size=0.15)

In [13]:
num_hid_layer1 = 512
num_hid_layer2 = 1024
num_hid_layer3 = 256
n_out = 1
n_in = X2.shape[1]
alpha = 0.0005
epochs = 3000
p1 = 0.9

In [14]:
with tf.device('/device:GPU:0'):
    X1_nn = tf.placeholder('float', [None, n_in])
    y1_nn = tf.placeholder('float', [None, n_out])
    weights1 = {'layer1':tf.Variable(tf.random_normal([n_in, num_hid_layer1])),
                'layer2':tf.Variable(tf.random_normal([num_hid_layer1, num_hid_layer2])),
                'layer3':tf.Variable(tf.random_normal([num_hid_layer2, num_hid_layer3])),
                'out':tf.Variable(tf.random_normal([num_hid_layer3, n_out]))
        
    }
    biases1 = {'layer1':tf.Variable(tf.random_normal([num_hid_layer1])),
               'layer2':tf.Variable(tf.random_normal([num_hid_layer2])),
               'layer3':tf.Variable(tf.random_normal([num_hid_layer3])),
               'out':tf.Variable(tf.random_normal([n_out]))
    }
    h1 = tf.add(tf.matmul(X1_nn, weights1['layer1']), biases1['layer1'])
    drop_con = tf.nn.dropout(h1, p1)*p1
    h2 = tf.add(tf.matmul(drop_con, weights1['layer2']), biases1['layer2'])
    h3 = tf.add(tf.matmul(h2, weights1['layer3']), biases1['layer3'])
    s = tf.add(tf.matmul(h3, weights1['out']), biases1['out'])
    loss_op1 = tf.losses.mean_squared_error(y1_nn, s)
    optimizer1 = tf.train.AdamOptimizer(alpha)
    train_op1 = optimizer1.minimize(loss_op1)
    init_op1 = tf.global_variables_initializer()
with tf.device('/device:CPU:0'):
    saver2 = tf.train.Saver()

In [ ]:
with tf.Session() as sess:
    sess.run(init_op1)
    for epoch in range(1, epochs+1):
        _ = sess.run(train_op1, feed_dict = {X1_nn:X2_tr, y1_nn:y_tr.reshape(y_tr.shape[0], 1)})
        if epoch%10 == 0:
            loss = sess.run(loss_op1, feed_dict = {X1_nn:X2_tr, y1_nn:y_tr.reshape(y_tr.shape[0], 1)})
            preds = sess.run(s, feed_dict = {X1_nn:X2_tr})
            print('Training:- after {} epochs, loss: {}, auc_score: {}'.format(epoch, loss, roc_auc_score(y_tr, preds)))
            loss = sess.run(loss_op1, feed_dict = {X1_nn:X2_ts, y1_nn:y_ts.reshape(y_ts.shape[0], 1)})
            preds = sess.run(s, feed_dict = {X1_nn:X2_ts})
            print('Test:- after {} epochs, loss: {}, auc_score: {}'.format(epoch, loss, roc_auc_score(y_ts, preds)))
    path2 = saver2.save(sess, 'E:/kaggle/home-credit-default-risk/dnn/neuralnetwork_selec_few.ckpt')
    print(path2)

In [ ]:
with tf.Session() as sess:
    saver2.restore(sess, path2)
    for epoch in range(1, 500+1):
        _ = sess.run(train_op1, feed_dict = {X1_nn:X2_tr, y1_nn:y_tr.reshape(y_tr.shape[0], 1)})
        if epoch%10 == 0:
            loss = sess.run(loss_op1, feed_dict = {X1_nn:X2_tr, y1_nn:y_tr.reshape(y_tr.shape[0], 1)})
            preds = sess.run(s, feed_dict = {X1_nn:X2_tr})
            print('Training:- after {} epochs, loss: {}, auc_score: {}'.format(epoch, loss, roc_auc_score(y_tr, preds)))
            loss = sess.run(loss_op1, feed_dict = {X1_nn:X2_ts, y1_nn:y_ts.reshape(y_ts.shape[0], 1)})
            preds = sess.run(s, feed_dict = {X1_nn:X2_ts})
            print('Test:- after {} epochs, loss: {}, auc_score: {}'.format(epoch, loss, roc_auc_score(y_ts, preds)))
    path2 = saver2.save(sess, 'E:/kaggle/home-credit-default-risk/dnn/neuralnetwork_selec_few.ckpt')

In [ ]:
with tf.Session() as sess:
    saver2.restore(sess, path2)
    for epoch in range(1, 500+1):
        _ = sess.run(train_op1, feed_dict = {X1_nn:X2_tr, y1_nn:y_tr.reshape(y_tr.shape[0], 1)})
        if 500%10 == 0:
            loss = sess.run(loss_op1, feed_dict = {X1_nn:X2_tr, y1_nn:y_tr.reshape(y_tr.shape[0], 1)})
            preds = sess.run(s, feed_dict = {X1_nn:X2_tr})
            print('Training:- after {} epochs, loss: {}, auc_score: {}'.format(epoch, loss, roc_auc_score(y_tr, preds)))
            loss = sess.run(loss_op1, feed_dict = {X1_nn:X2_ts, y1_nn:y_ts.reshape(y_ts.shape[0], 1)})
            preds = sess.run(s, feed_dict = {X1_nn:X2_ts})
            print('Test:- after {} epochs, loss: {}, auc_score: {}'.format(epoch, loss, roc_auc_score(y_ts, preds)))
    path2 = saver2.save(sess, 'E:/kaggle/home-credit-default-risk/dnn/neuralnetwork_selec_few.ckpt')

In [ ]:
with tf.Session() as sess:
    saver2.restore(sess, path2)
    for epoch in range(1, 1000+1):
        _ = sess.run(train_op1, feed_dict = {X1_nn:X2_tr, y1_nn:y_tr.reshape(y_tr.shape[0], 1)})
        if 500%10 == 0:
            loss = sess.run(loss_op1, feed_dict = {X1_nn:X2_tr, y1_nn:y_tr.reshape(y_tr.shape[0], 1)})
            preds = sess.run(s, feed_dict = {X1_nn:X2_tr})
            print('Training:- after {} epochs, loss: {}, auc_score: {}'.format(epoch, loss, roc_auc_score(y_tr, preds)))
            loss = sess.run(loss_op1, feed_dict = {X1_nn:X2_ts, y1_nn:y_ts.reshape(y_ts.shape[0], 1)})
            preds = sess.run(s, feed_dict = {X1_nn:X2_ts})
            print('Test:- after {} epochs, loss: {}, auc_score: {}'.format(epoch, loss, roc_auc_score(y_ts, preds)))
    path2 = saver2.save(sess, 'E:/kaggle/home-credit-default-risk/dnn/neuralnetwork_selec_few.ckpt')

In [15]:
train_probs = pd.read_csv('train_probs.csv')

In [16]:
test_probs = pd.read_csv('test_probs.csv')

In [17]:
submission_probs = pd.read_csv('submission_probs.csv')

In [31]:
with tf.Session() as sess:
    saver2.restore(sess, 'E:/kaggle/home-credit-default-risk/dnn/neuralnetwork_selec_few.ckpt')
    preds = sess.run(s, feed_dict={X1_nn:X2_tr})
    print(roc_auc_score(y_tr, preds))
    train_probs['nn_less'] = preds

INFO:tensorflow:Restoring parameters from E:/kaggle/home-credit-default-risk/dnn/neuralnetwork_selec_few.ckpt
0.5667603916269951


In [32]:
with tf.Session() as sess:
    saver2.restore(sess, 'E:/kaggle/home-credit-default-risk/dnn/neuralnetwork_selec_few.ckpt')
    preds = sess.run(s, feed_dict={X1_nn:X2_ts})
    print(roc_auc_score(y_ts, preds))
    test_probs['nn_less'] = preds

INFO:tensorflow:Restoring parameters from E:/kaggle/home-credit-default-risk/dnn/neuralnetwork_selec_few.ckpt
0.5608011743500125


In [29]:
with tf.Session() as sess:
    saver2.restore(sess, 'E:/kaggle/home-credit-default-risk/dnn/neuralnetwork_selec_few.ckpt')
    preds = sess.run(s, feed_dict={X1_nn:X2_sub})
    submission_probs['nn_less'] = preds

INFO:tensorflow:Restoring parameters from E:/kaggle/home-credit-default-risk/dnn/neuralnetwork_selec_few.ckpt


In [30]:
train_probs.tail()

,Unnamed: 0,best_rf,simple_xgbc,xgbc_selec_cv,xgbc_less_cv,nn_selec,nn_full,nn_less
261379,402535,0.055408,0.046214,0.021470,0.163618,0.123099,-0.072081,-13784.418945
261380,402537,0.138368,0.067537,0.055454,0.074119,0.140188,0.216982,26009.480469
261381,402538,0.067350,0.039181,0.033557,0.113988,0.158743,0.046083,9699.157227
261382,402539,0.051280,0.191010,0.180102,0.008170,0.024954,0.028505,-2246.506592
261383,402540,0.241888,0.115924,0.100112,0.037956,0.023978,0.006532,-95605.765625


In [44]:
def softmax(x):
    s = np.exp(x)
    return s/np.sum(s)

In [45]:
x = np.concatenate((train_probs['nn_less'].values, test_probs['nn_less'].values, submission_probs['nn_less'].values), axis=0)

In [46]:
x = softmax(x)

c:\python\python36\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
c:\python\python36\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
x

array([ 0.,  0., nan, ..., nan, nan, nan], dtype=float32)

In [48]:
roc_auc_score(y_tr, x[:y_tr.shape[0]])

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
submission.to_csv('submission_nn_selec_few.csv', index=False)